# JSON Extraction
Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
* Find the 10 countries with most projects
* Find the top 10 major project themes (using column 'mjtheme_namecode')

In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [4]:
import os
os.getcwd()
path = 'C:/Users/jdavo/Documents/data_wrangling_json/data_wrangling_json/data'
os.chdir(path)
os.getcwd()

'C:\\Users\\jdavo\\Documents\\data_wrangling_json\\data_wrangling_json\\data'

In [393]:
import pandas as pd
import json
from pandas.io.json import json_normalize
data = pd.read_json('world_bank_projects.json')
#print(data.describe())
#print(data.info())
data.set_index('id', inplace=True)
data.iloc[:,41:50].head()

,status,supplementprojectflg,theme1,theme_namecode,themecode,totalamt,totalcommamt,url
id,,,,,,,,
P129828,Active,N,"{'Percent': 100, 'Name': 'Education for all'}","[{'code': '65', 'name': 'Education for all'}]",65,130000000,130000000,http://www.worldbank.org/projects/P129828/ethi...
P144674,Active,N,"{'Percent': 30, 'Name': 'Other economic manage...","[{'code': '24', 'name': 'Other economic manage...","54,24",0,4700000,http://www.worldbank.org/projects/P144674?lang=en
P145310,Active,Y,"{'Percent': 46, 'Name': 'Regional integration'}","[{'code': '47', 'name': 'Regional integration'...","52,81,25,47",6060000,6060000,http://www.worldbank.org/projects/P145310?lang=en
P144665,Active,N,"{'Percent': 50, 'Name': 'Participation and civ...","[{'code': '57', 'name': 'Participation and civ...","59,57",0,1500000,http://www.worldbank.org/projects/P144665?lang=en
P144933,Active,N,"{'Percent': 30, 'Name': 'Export development an...","[{'code': '45', 'name': 'Export development an...","41,45",13100000,13100000,http://www.worldbank.org/projects/P144933/seco...


In [265]:
# Cleaning:
data['country_namecode'] = data['country_namecode'].apply(lambda x: x.split('!$!')[1])

In [267]:
data.groupby('countryname')._id.count().sort_values(ascending=False).head(10)

countryname
People's Republic of China         19
Republic of Indonesia              19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
Nepal                              12
People's Republic of Bangladesh    12
Kingdom of Morocco                 12
Africa                             11
Republic of Mozambique             11
Name: _id, dtype: int64

In [269]:
pd.Series(tot_dict).groupby(tot_dict).count().sort_values(ascending=False)

Environment and natural resources management    223
Rural development                               202
Human development                               197
Public sector governance                        184
Social protection and risk management           158
Financial and private sector development        130
                                                122
Social dev/gender/inclusion                     119
Trade and integration                            72
Urban development                                47
Economic management                              33
Rule of law                                      12
dtype: int64

In [270]:
tot_dict = []
bs = data['mjtheme_namecode'].apply(lambda x: json_normalize(x))
for b in bs:
    for a in range(len(b['name'])):
        tot_dict.append(b['name'][a])
            
themes = pd.Series(tot_dict).groupby(tot_dict).count().sort_values(ascending=False)
themes = themes.reset_index()
theme10 = themes[themes['index'] != ''].head(10)
theme10.columns = ['Theme', 'Total']
theme10

,Theme,Total
0,Environment and natural resources management,223
1,Rural development,202
2,Human development,197
3,Public sector governance,184
4,Social protection and risk management,158
5,Financial and private sector development,130
7,Social dev/gender/inclusion,119
8,Trade and integration,72
9,Urban development,47
10,Economic management,33


In [279]:
data['majorsector_percent'].apply(lambda x: json_normalize(x))[0]


,Name,Percent
0,Education,46
1,Education,26
2,"Public Administration, Law, and Justice",16
3,Education,12


In [384]:
new = []
bs = data['majorsector_percent'].apply(lambda x: json_normalize(x))
for b in range(len(bs)):
    for a in range(len(bs[b])):
        new.append([bs[b]['Name'][a], bs[b]['Percent'][a], data.index[b]])
new = pd.DataFrame(new).set_index([2,0])
new.index.names=['ID', 'Major Sector']; new.columns = ['Percent [%]']
new.head(15)

Percent [%]
ID      Major Sector                                        
P129828 Education                                         46
        Education                                         26
        Public Administration, Law, and Justice           16
        Education                                         12
P144674 Public Administration, Law, and Justice           70
        Public Administration, Law, and Justice           30
P145310 Transportation                                   100
P144665 Health and other social services                 100
P144933 Industry and trade                                50
        Industry and trade                                40
        Finance                                           10
P146161 Health and other social services                 100
P121185 Transportation                                   100
P127033 Energy and mining                                100
P130164 Transportation                                   100

In [383]:
new2 = []
bs = data['sector'].apply(lambda x: json_normalize(x))
for b in range(len(bs)):
    for a in range(len(bs[b])):
        new2.append([bs[b]['Name'][a], data.index[b]])
new2 = pd.DataFrame(new2).set_index([1,0])
new2.index.names = ['ID', 'Sector']
new2.head(15)

Empty DataFrame
Columns: []
Index: [(P129828, Primary education), (P129828, Secondary education), (P129828, Public administration- Other social services), (P129828, Tertiary education), (P144674, Public administration- Other social services), (P144674, General public administration sector), (P145310, Rural and Inter-Urban Roads and Highways), (P144665, Other social services), (P144933, General industry and trade sector), (P144933, Other industry), (P144933, SME Finance), (P146161, Other social services), (P121185, Rural and Inter-Urban Roads and Highways), (P127033, Other Renewable Energy), (P130164, Rural and Inter-Urban Roads and Highways)]

In [434]:
def mindex_merge(self, df, on):
    return self.reset_index().merge(df.reset_index(), on=on, left_index=True, right_index=True)
print(new2.reset_index().head())
mindex_merge(new2, new, 'ID').set_index(['ID', 'Major Sector'])

        ID                                        Sector
0  P129828                             Primary education
1  P129828                           Secondary education
2  P129828  Public administration- Other social services
3  P129828                            Tertiary education
4  P144674  Public administration- Other social services


Sector  \
ID      Major Sector                                                                                 
P129828 Education                                                                Primary education   
        Education                                                              Secondary education   
        Public Administration, Law, and Justice       Public administration- Other social services   
        Education                                                               Tertiary education   
P144674 Public Administration, Law, and Justice       Public administration- Other social services   
        Public Administration, Law, and Justice               General public administration sector   
P145310 Transportation                                    Rural and Inter-Urban Roads and Highways   
P144665 Health and other social services                                     Other social services   
P144933 Industry and trade                                       General industry and trade sector   
        Industry and trade                                                          Other industry   
        Finance                                                                        SME Finance   
P146161 Health and other social services                                     Other social services   
P121185 Transportation                                    Rural and Inter-Urban Roads and Highways   
P127033 Energy and mining                                                   Other Renewable Energy   
P130164 Transportation                                    Rural and Inter-Urban Roads and Highways   
P130903 Public Administration, Law, and Justice               General public administration sector   
        Public Administration, Law, and Justice                  Central government administration   
        Public Administration, Law, and Justice  Public administration- Information and communi...   
P145339 Agriculture, fishing, and forestry                                                   Crops   
        Health and other social services                                     Other social services   
        Agriculture, fishing, and forestry        General agriculture, fishing and forestry sector   
P146653 Transportation                                    Rural and Inter-Urban Roads and Highways   
        Water, sanitation and flood protection                                    Flood protection   
        Industry and trade                                                    Housing construction   
        Health and other social services                                     Other social services   
P144140 Information and communications               General information and communications sector   
P130975 Transportation                                    Rural and Inter-Urban Roads and Highways   
        Public Administration, Law, and Justice              Public administration- Transportation   
P144832 Health and other social services                                     Other social services   
P145545 Transportation                                    Rural and Inter-Urban Roads and Highways   
...                                                                                            ...   
P132097 Public Administration, Law, and Justice       Public administration- Other social services   
        Public Administration, Law, and Justice               General public administration sector   
P121116 Agriculture, fishing, and forestry                     Agricultural extension and research   
        Public Administration, Law, and Justice  Public administration- Agriculture, fishing an...   
        Industry and trade                                     Agro-industry, marketing, and trade   
P126465 Public Administration, Law, and Justice             Sub-national government administration   
        Education                                                         General education sector   
        Heal